In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('once')
from sklearn.tree import DecisionTreeRegressor
import datetime as dt

In [31]:
df = pd.read_csv('../datos/casual/15-casual-cod_est.csv', index_col=0)
df.head(2)

,temp,atemp,hum,windspeed,season,holiday,weekday,weathersit,temp.1,casual
0,-0.486274,-0.456399,0.855582,-0.208909,0,0,3,1,14.110847,331
1,-0.425690,-0.492863,0.332722,0.687550,0,2,1,1,14.902598,131


In [32]:
df.dtypes

temp          float64
atemp         float64
hum           float64
windspeed     float64
season          int64
holiday         int64
weekday         int64
weathersit      int64
temp.1        float64
casual          int64
dtype: object

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 729
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   temp        730 non-null    float64
 1   atemp       730 non-null    float64
 2   hum         730 non-null    float64
 3   windspeed   730 non-null    float64
 4   season      730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   weathersit  730 non-null    int64  
 8   temp.1      730 non-null    float64
 9   casual      730 non-null    int64  
dtypes: float64(5), int64(5)
memory usage: 62.7 KB


In [34]:
df.head(2)

,temp,atemp,hum,windspeed,season,holiday,weekday,weathersit,temp.1,casual
0,-0.486274,-0.456399,0.855582,-0.208909,0,0,3,1,14.110847,331
1,-0.425690,-0.492863,0.332722,0.687550,0,2,1,1,14.902598,131


---

In [35]:
X = df.drop("casual", axis = 1)
y = df["casual"]

In [36]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42) 

In [37]:
arbol = DecisionTreeRegressor(random_state =0)
arbol.fit(x_train, y_train)

DecisionTreeRegressor(random_state=0)

In [38]:
#fig = plt.figure(figsize = (10,6))
#tree.plot_tree(arbol, feature_names = x_train.columns, filled = True)
#plt.show()

In [39]:
max_features = np.sqrt(len(x_train.columns))
max_features

3.0

In [40]:
print(arbol.tree_.max_depth)

25


In [41]:
param = {"max_depth": [4,6,8,10],
        "max_features": [1,2,3],
        "min_samples_split": [10,40,70],
        "min_samples_leaf": [10,40,70]} 

In [42]:
gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), 
            param_grid= param, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error")
gs.fit(x_train, y_train)



GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [4, 6, 8, 10], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 40, 70],
                         'min_samples_split': [10, 40, 70]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [43]:
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=10, max_features=3, min_samples_leaf=10,
                      min_samples_split=40)

In [44]:
#fig = plt.figure(figsize=(40, 20))
#tree.plot_tree(mejor_modelo, feature_names=x_train.columns, filled=True);

In [45]:
y_pred_test_dt2 = mejor_modelo.predict(x_test)
y_pred_train_dt2 = mejor_modelo.predict(x_train)

In [46]:

def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [47]:
dt_results1 = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "1_Dtree_casual_est-cod")
dt_results1

,MAE,MSE,RMSE,R2,set,modelo
0,321.116696,218687.564917,467.640423,0.544803,test,1_Dtree_casual_est-cod
1,274.762222,165665.210704,407.019914,0.645792,train,1_Dtree_casual_est-cod


Random forest

In [92]:
X = df.drop("casual", axis = 1)
y = df["casual"]

In [93]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [94]:
param = {"max_depth": [6,8,10,12,14],
        "max_features": [1,2,3],
        "min_samples_split": [10,30,60,90],
        "min_samples_leaf": [10,30,60,90]} 

In [95]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), 
            param_grid= param, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 


In [96]:
gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [6, 8, 10, 12, 14],
                         'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 30, 60, 90],
                         'min_samples_split': [10, 30, 60, 90]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [97]:
bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=8, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

In [98]:
'''for arbol in tqdm(bosque.estimators_):
    fig = plt.figure(figsize=(40, 20))
    tree.plot_tree(arbol, feature_names= x_train.columns, filled = True)
    plt.show()'''

'for arbol in tqdm(bosque.estimators_):\n    fig = plt.figure(figsize=(40, 20))\n    tree.plot_tree(arbol, feature_names= x_train.columns, filled = True)\n    plt.show()'

In [99]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
   
   
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df


In [100]:
dt_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "2_RForest_casual_cod_estand")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,294.875349,176849.600499,420.534898,0.631889,test,2_RForest_casual_cod_estand
1,267.737272,150213.806292,387.574259,0.678829,train,2_RForest_casual_cod_estand


In [101]:
df_todos_resultados = pd.concat([dt_results, dt_results1], axis = 0)
df_todos_resultados


,MAE,MSE,RMSE,R2,set,modelo
0,294.875349,176849.600499,420.534898,0.631889,test,2_RForest_casual_cod_estand
1,267.737272,150213.806292,387.574259,0.678829,train,2_RForest_casual_cod_estand
0,321.116696,218687.564917,467.640423,0.544803,test,1_Dtree_casual_est-cod
1,274.762222,165665.210704,407.019914,0.645792,train,1_Dtree_casual_est-cod


In [102]:
df_todos_resultados.to_csv('../datos/casual/16-casual-metricas-estand.csv')

# Random Forest es la mejor opcion